## Load Data and Tokenizer

In [ ]:
from transformers import BertTokenizer
from datasets import load_dataset

ds = load_dataset('openpecha/tagged_cleaned_MT_v1.0.3')

tokenizer = BertTokenizer.from_pretrained('tibetan_tokenizer')

ds['train'][0]

## Preprocess Data

### Remove Blank Tags

In [ ]:
def condition(example):
    return example['Tag'] != ''

ds = ds.filter(condition)

In [ ]:
ds

In [ ]:
ds['train'][0]

### Collapse Buddhist Lables into One

In [ ]:
buddhist_labels = ['Mantras',
                    'Dzogchen',
                    'Astrology',
                    'Monastery',
                    'Mahamudra',
                    'Mind',
                    'Meditation',
                    'Self, Logic, Aggregates',
                    'Tantra',
                    'Emptiness',
                    'Dreams',
                    'Education, Teaching',
                    'Ethics, Enlightenment, Wisdom',
                    'Prophecies, Rituals',
                    'Lama',
                    'Samsara, Nirvana',
                    'Milarepa, Realization, Biography',
                    'Kayas',
                    'Intrinsic Existence, Conventional Existence',
                    'Time, Causality, Perception',
                    'Natural State',
                    'Karma, Consequences',
                    'Dharma']

In [ ]:
def collapse_labels(example):
    if example['Tag'] in buddhist_labels:
        example['Tag'] = 'Buddhist'
    return example

# Apply the function to the dataset
ds = ds.map(collapse_labels)

In [ ]:
ds

### Convert Labels to Id Numbers

In [ ]:
all_tags = list(set(ds['train']['Tag']))

# Create a label-to-index mapping
label2id = {label: idx for idx, label in enumerate(all_tags)}
id2label = {idx: label for label, idx in label2id.items()}

# Save label mappings for future use
import json
with open("simple_op_label_mapping.json", "w") as f:
    json.dump(label2id, f)


In [ ]:
def preprocess(examples):
    tokens = tokenizer(examples["Source"], padding="max_length", truncation=True, max_length=128)
    tokens["labels"] = [label2id[label] for label in examples["Tag"]]    
    return tokens

encoded_dataset = ds.map(preprocess, batched=True)


In [ ]:
encoded_dataset = encoded_dataset.remove_columns(['Source', 'Target', 'File_Name', 'Machine Aligned', '__index_level_0__', 'Tag'])

In [ ]:
encoded_dataset = encoded_dataset['train'].train_test_split(.15)

## Train Model

In [ ]:
from transformers import BertForSequenceClassification

# Load tokenizer and model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label2id))

# Resize embeddings to match the new tokenizer
model.resize_token_embeddings(len(tokenizer))

# Move model to GPU
model = model.to('cuda:0')

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    predictions, references = eval_pred
    
    # Get predicted class indices
    predictions = np.argmax(predictions, axis=1)
    
    # Compute metrics
    accuracy = accuracy_score(references, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(references, predictions, average="weighted")
    
    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }


In [ ]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

# Define training arguments
training_args = TrainingArguments(
    output_dir="fixed-col-op-bert-classifier",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=100,  # Set a maximum number of epochs
    weight_decay=0.01,
    eval_strategy="epoch",  # Evaluate at the end of every epoch
    save_strategy="epoch",  # Save the model at the end of every epoch
    load_best_model_at_end=True,  # Load the best model after training
    metric_for_best_model="accuracy",  # Metric to monitor
    greater_is_better=True,  # Higher accuracy is better
    logging_dir="./logs"
)

# Add the EarlyStoppingCallback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3  # Stop training if the metric does not improve for 3 evaluation steps
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]  # Add the early stopping callback
)

# Start training
trainer.train()